In [23]:
from sympy import sin, cos, Matrix, Symbol, solve, simplify, pi
import math

In [24]:
# 脚毎の「脚根本ー胴体重心変換行列」を定義
#pi=3.14159
Phi = 0
Theta = pi/2.0-0.955316618
Psi = 3*pi/4.0
body = 0.1

R = Matrix([
    [cos(Theta)*cos(Psi), sin(Phi)*sin(Theta)*cos(Psi)-cos(Phi)*sin(Psi), cos(Phi)*sin(Theta)*cos(Psi)+sin(Phi)*sin(Psi)],
    [cos(Theta)*sin(Psi), sin(Phi)*sin(Theta)*sin(Psi)+cos(Phi)*cos(Psi), cos(Phi)*sin(Theta)*sin(Psi)-sin(Phi)*cos(Psi)],
    [-sin(Theta), sin(Phi)*cos(Theta), cos(Phi)*cos(Theta)]
])
# 胴体中心から脚根本までの座標
# BLF
col = Matrix([[-body/2.0],
            [body/2.0],
            [-body/2.0]])
row = Matrix([[0, 0, 0, 1]])
T_tmp = R.col_insert(4, col)
convert_BLF_to_center = T_tmp.row_insert(4, row)
print(convert_BLF_to_center)

Matrix([[cos(0.75*pi)*cos(-0.5*pi + 0.955316618), -sin(0.75*pi), -sin(-0.5*pi + 0.955316618)*cos(0.75*pi), -0.0500000000000000], [sin(0.75*pi)*cos(-0.5*pi + 0.955316618), cos(0.75*pi), -sin(0.75*pi)*sin(-0.5*pi + 0.955316618), 0.0500000000000000], [sin(-0.5*pi + 0.955316618), 0, cos(-0.5*pi + 0.955316618), -0.0500000000000000], [0, 0, 0, 1]])


In [25]:
# 回転軸の方向ベクトル
# 元座標からみた相対座標系における回転軸方向のベクトルを表す
class Omega:
    def __init__(self, Lam=0, Mu=0, Nu=0, Phi=0, Theta=0, Psi=0):
        self.Lam, self.Mu, self.Nu, self.Phi, self.Theta, self.Psi = Lam, Mu, Nu, Phi, Theta, Psi
    def subsVal(self, Lam, Mu, Nu):
        self.Lam, self.Mu, self.Nu = Lam, Mu, Nu
    def subsEuler(self, Phi, Theta, Psi):
        self.Phi, self.Theta, self.Psi = Phi, Theta, Psi

# オイラー回転を行った際の, ｚ軸方向ベクトルの回転後ベクトルを計算
def convertEulerToOmega(Phi, Theta, Psi):
        Lam = cos(Phi)*sin(Theta)*cos(Psi)+sin(Phi)*sin(Psi)
        Mu = cos(Phi)*sin(Theta)*sin(Psi)-sin(Phi)*cos(Psi)
        Nu = cos(Phi)*cos(Theta)
        print(Lam, Mu, Nu)
        return Lam, Mu, Nu

# 同時変換行列の導出
def getSimultaneousTransformationXMatrix(omega, theta, L):
    X_roll = Matrix([[1, 0, 0],
                [0, cos(theta), -sin(theta)],
                [0, sin(theta), cos(theta)]])
    RzRyRz = Matrix([
                [cos(omega.Theta)*cos(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)-cos(omega.Phi)*sin(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)+sin(omega.Phi)*sin(omega.Psi)],
                [cos(omega.Theta)*sin(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)+cos(omega.Phi)*cos(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)-sin(omega.Phi)*cos(omega.Psi)],
                [-sin(omega.Theta), sin(omega.Phi)*cos(omega.Theta), cos(omega.Phi)*cos(omega.Theta)]])
    R = RzRyRz * X_roll
    col = Matrix([[L],
                [0],
                [0]])
    row = Matrix([[0, 0, 0, 1]])
    T_tmp = R.col_insert(4, col)
    T = T_tmp.row_insert(4, row)
    return T

def getSimultaneousTransformationZMatrix(omega, theta, L):
    Z_roll = Matrix([[cos(theta), -sin(theta), 0],
                [sin(theta), cos(theta), 0],
                [0, 0, 1]])
    RzRyRz = Matrix([
                [cos(omega.Theta)*cos(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)-cos(omega.Phi)*sin(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*cos(omega.Psi)+sin(omega.Phi)*sin(omega.Psi)],
                [cos(omega.Theta)*sin(omega.Psi), sin(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)+cos(omega.Phi)*cos(omega.Psi), cos(omega.Phi)*sin(omega.Theta)*sin(omega.Psi)-sin(omega.Phi)*cos(omega.Psi)],
                [-sin(omega.Theta), sin(omega.Phi)*cos(omega.Theta), cos(omega.Phi)*cos(omega.Theta)]])
    R = RzRyRz * Z_roll
    col = Matrix([[L],
                [0],
                [0]])
    row = Matrix([[0, 0, 0, 1]])
    T_tmp = R.col_insert(4, col)
    T = T_tmp.row_insert(4, row)
    return T

# 合成同時変換行列の導出
def calcCompositeSimultaneousTransformationMatrix(omega, L, theta):
    T_0 = getSimultaneousTransformationXMatrix(omega[0], theta = theta[0], L = L[0])
    T_1 = getSimultaneousTransformationZMatrix(omega[1], theta = theta[1], L = L[1])
    T_2 = getSimultaneousTransformationZMatrix(omega[2], theta = theta[2], L = L[2])
    Theta = Matrix([theta[0], theta[1], theta[2]])
    P_3 = Matrix([[L[3]],
                [0],
                [0],
                [1]])
    #P =  convert_BLF_to_center*T_0 * T_1 * T_2 * P_3
    P =  T_0 * T_1 * T_2 * P_3
    Jacob_T = P.jacobian(Theta)
    # 4行目を消去
    P.row_del(3)
    Jacob_T.row_del(3)
    return Jacob_T, P

In [26]:
theta = [Symbol('theta[0]', real=True), Symbol('theta[1]', real=True), Symbol('theta[2]', real=True)]
euler_0 = [0, 0, 0]
euler_1 = [-pi/2, 0, 0]
euler_2 = [0, 0, 0]
L = [0, Symbol('self.__L[1]', real=True), Symbol('self.__L[2]', real=True), Symbol('self.__L[3]', real=True)]
L = [0.0, 0.03, 0.1, 0.1]
omega = [Omega(), Omega(), Omega()]
omega[0].subsEuler(euler_0[0], euler_0[1], euler_0[2])
omega[1].subsEuler(euler_1[0], euler_1[1], euler_1[2])
omega[2].subsEuler(euler_2[0], euler_2[1], euler_2[2])

for i in range(3):
    convertEulerToOmega(omega[i].Phi, omega[i].Theta, omega[i].Psi)

# ヤコビ行列, 胴体座標
Jacob_T, P = calcCompositeSimultaneousTransformationMatrix(omega, L, theta)
#print(Jacob_T)
print(P)

0 0 1
0 1 0
0 0 1
Matrix([[-0.1*sin(theta[1])*sin(theta[2]) + 0.1*cos(theta[1])*cos(theta[2]) + 0.1*cos(theta[1]) + 0.03], [0.1*sin(theta[0])*sin(theta[1])*cos(theta[2]) + 0.1*sin(theta[0])*sin(theta[1]) + 0.1*sin(theta[0])*sin(theta[2])*cos(theta[1])], [-0.1*sin(theta[1])*cos(theta[0])*cos(theta[2]) - 0.1*sin(theta[1])*cos(theta[0]) - 0.1*sin(theta[2])*cos(theta[0])*cos(theta[1])]])


In [32]:
now_theta = Matrix([0, -60.0*pi/180.0, 120.0*pi/180.0])

now_pos = P.subs([(theta[0], now_theta[0]), (theta[1], now_theta[1]), (theta[2], now_theta[2])])
print(now_pos.evalf(4))
target_pos = Matrix([0.12, 0.05, -0.10])
repeat = 1000
for i in range(repeat):
    # 胴体座標形における足先座標の計算
    ABS_pos = P.subs([(theta[0], now_theta[0]), (theta[1], now_theta[1]), (theta[2], now_theta[2])])
    print(ABS_pos.evalf())
    # 座標フィードバックをもとにして, 脚先の目標方向ベクトルの導出
    target_pos_vector = now_pos+(target_pos-now_pos)*(float(i+1) /  float(repeat)) 
    #print((ABS_pos+target_pos_vector).evalf(4))
    # ヤコビ行列の逆行列の計算
    J = Jacob_T.subs([(theta[0], now_theta[0]), (theta[1], now_theta[1]), (theta[2], now_theta[2])])
    #print(J)
    inv_J = J.inv()
    delta_pos = target_pos_vector - ABS_pos
    delta_theta = inv_J*delta_pos
    # 目標方向ベクトル, 現在関節角度をもとにして目標関節角度を更新
    target_theta = (now_theta + delta_theta).evalf()

    now_theta = target_theta

Matrix([[0.1300], [0], [0.e-120]])
Matrix([[0.130000000000000], [0], [-0.e-125]])
Matrix([[0.129989949838343], [6.66557083797460e-5], [-7.45319614286477e-5]])
Matrix([[0.129979932846972], [0.000143810607170884], [-0.000160803731401753]])
Matrix([[0.129969941644525], [0.000215720311663989], [-0.000241210518035703]])
Matrix([[0.129959941643089], [0.000287629475295714], [-0.000321616699898358]])
Matrix([[0.129949941641007], [0.000359538645259075], [-0.000402022888840808]])
Matrix([[0.129939941638888], [0.000431447815275980], [-0.000482429077843141]])
Matrix([[0.129929941636724], [0.000503356985344781], [-0.000562835266903490]])
Matrix([[0.129919941634514], [0.000575266155464754], [-0.000643241456021071]])
Matrix([[0.129909941632258], [0.000647175325635303], [-0.000723647645195190]])
Matrix([[0.129899941629956], [0.000719084495855782], [-0.000804053834425146]])
Matrix([[0.129889941627607], [0.000790993666125572], [-0.000884460023710239]])
Matrix([[0.129879941625212], [0.000862902836443916]

Matrix([[0.128909941169058], [0.00783809248195624], [-0.00876426671832987]])
Matrix([[0.128899941162107], [0.00791000165372027], [-0.00884467290928578]])
Matrix([[0.128889941155112], [0.00798191082546448], [-0.00892507910021951]])
Matrix([[0.128879941148075], [0.00805381999718820], [-0.00900548529113033]])
Matrix([[0.128869941140996], [0.00812572916889068], [-0.00908589148201741]])
Matrix([[0.128859941133874], [0.00819763834057128], [-0.00916629767288001]])
Matrix([[0.128849941126710], [0.00826954751222929], [-0.00924670386371736]])
Matrix([[0.128839941119505], [0.00834145668386399], [-0.00932711005452865]])
Matrix([[0.128829941112257], [0.00841336585547477], [-0.00940751624531319]])
Matrix([[0.128819941104969], [0.00848527502706089], [-0.00948792243607014]])
Matrix([[0.128809941097638], [0.00855718419862160], [-0.00956832862679872]])
Matrix([[0.128799941090267], [0.00862909337015634], [-0.00964873481749821]])
Matrix([[0.128789941082854], [0.00870100254166435], [-0.00972914100816783]])

Matrix([[0.127819940204157], [0.0156761919484838], [-0.0175285412465505]])
Matrix([[0.127809940193889], [0.0157481011142733], [-0.0176089474308259]])
Matrix([[0.127799940183606], [0.0158200102799758], [-0.0176893536150041]])
Matrix([[0.127789940173309], [0.0158919194455910], [-0.0177697597990846]])
Matrix([[0.127779940162998], [0.0159638286111183], [-0.0178501659830668]])
Matrix([[0.127769940152674], [0.0160357377765573], [-0.0179305721669502]])
Matrix([[0.127759940142337], [0.0161076469419073], [-0.0180109783507342]])
Matrix([[0.127749940131986], [0.0161795561071679], [-0.0180913845344182]])
Matrix([[0.127739940121623], [0.0162514652723388], [-0.0181717907180018]])
Matrix([[0.127729940111248], [0.0163233744374192], [-0.0182521969014844]])
Matrix([[0.127719940100862], [0.0163952836024089], [-0.0183326030848655]])
Matrix([[0.127709940090463], [0.0164671927673072], [-0.0184130092681444]])
Matrix([[0.127699940080054], [0.0165391019321138], [-0.0184934154513207]])
Matrix([[0.12768994006963

Matrix([[0.126709939052716], [0.0236581087262017], [-0.0264536270023609]])
Matrix([[0.126699939042960], [0.0237300178799111], [-0.0265340331731287]])
Matrix([[0.126689939033228], [0.0238019270334952], [-0.0266144393437565]])
Matrix([[0.126679939023521], [0.0238738361869541], [-0.0266948455142442]])
Matrix([[0.126669939013837], [0.0239457453402874], [-0.0267752516845916]])
Matrix([[0.126659939004179], [0.0240176544934950], [-0.0268556578547984]])
Matrix([[0.126649938994546], [0.0240895636465769], [-0.0269360640248646]])
Matrix([[0.126639938984938], [0.0241614727995326], [-0.0270164701947897]])
Matrix([[0.126629938975356], [0.0242333819523622], [-0.0270968763645738]])
Matrix([[0.126619938965800], [0.0243052911050654], [-0.0271772825342166]])
Matrix([[0.126609938956271], [0.0243772002576421], [-0.0272576887037179]])
Matrix([[0.126599938946768], [0.0244491094100921], [-0.0273380948730776]])
Matrix([[0.126589938937293], [0.0245210185624153], [-0.0274185010422954]])
Matrix([[0.12657993892784

Matrix([[0.125609938185169], [0.0315681148603673], [-0.0352983049215158]])
Matrix([[0.125599938179694], [0.0316400239997939], [-0.0353787110763132]])
Matrix([[0.125589938174270], [0.0317119331390908], [-0.0354591172309655]])
Matrix([[0.125579938168897], [0.0317838422782580], [-0.0355395233854728]])
Matrix([[0.125569938163576], [0.0318557514172956], [-0.0356199295398352]])
Matrix([[0.125559938158306], [0.0319276605562037], [-0.0357003356940528]])
Matrix([[0.125549938153089], [0.0319995696949824], [-0.0357807418481257]])
Matrix([[0.125539938147923], [0.0320714788336317], [-0.0358611480020540]])
Matrix([[0.125529938142809], [0.0321433879721518], [-0.0359415541558377]])
Matrix([[0.125519938137747], [0.0322152971105427], [-0.0360219603094771]])
Matrix([[0.125509938132737], [0.0322872062488046], [-0.0361023664629721]])
Matrix([[0.125499938127780], [0.0323591153869375], [-0.0361827726163229]])
Matrix([[0.125489938122876], [0.0324310245249415], [-0.0362631787695296]])
Matrix([[0.12547993811802

Matrix([[0.124509937912283], [0.0394781194453004], [-0.0441429811083759]])
Matrix([[0.124499937912967], [0.0395500285712405], [-0.0442233872480932]])
Matrix([[0.124489937913708], [0.0396219376970674], [-0.0443037933876839]])
Matrix([[0.124479937914506], [0.0396938468227814], [-0.0443841995271483]])
Matrix([[0.124469937915361], [0.0397657559483827], [-0.0444646056664867]])
Matrix([[0.124459937916273], [0.0398376650738714], [-0.0445450118056993]])
Matrix([[0.124449937917243], [0.0399095741992478], [-0.0446254179447862]])
Matrix([[0.124439937918269], [0.0399814833245121], [-0.0447058240837478]])
Matrix([[0.124429937919353], [0.0400533924496644], [-0.0447862302225842]])
Matrix([[0.124419937920493], [0.0401253015747050], [-0.0448666363612957]])
Matrix([[0.124409937921691], [0.0401972106996341], [-0.0449470424998825]])
Matrix([[0.124399937922945], [0.0402691198244518], [-0.0450274486383448]])
Matrix([[0.124389937924256], [0.0403410289491584], [-0.0451078547766828]])
Matrix([[0.12437993792562

Matrix([[0.123389938326360], [0.0475319408937822], [-0.0531484680222894]])
Matrix([[0.123379938332874], [0.0476038500083163], [-0.0532288741492529]])
Matrix([[0.123369938339433], [0.0476757591227598], [-0.0533092802761151]])
Matrix([[0.123359938346035], [0.0477476682371129], [-0.0533896864028761]])
Matrix([[0.123349938352680], [0.0478195773513757], [-0.0534700925295364]])
Matrix([[0.123339938359369], [0.0478914864655485], [-0.0535504986560959]])
Matrix([[0.123329938366102], [0.0479633955796315], [-0.0536309047825549]])
Matrix([[0.123319938372877], [0.0480353046936248], [-0.0537113109089137]])
Matrix([[0.123309938379696], [0.0481072138075286], [-0.0537917170351724]])
Matrix([[0.123299938386557], [0.0481791229213430], [-0.0538721231613312]])
Matrix([[0.123289938393460], [0.0482510320350684], [-0.0539525292873904]])
Matrix([[0.123279938400406], [0.0483229411487048], [-0.0540329354133501]])
Matrix([[0.123269938407394], [0.0483948502622524], [-0.0541133415392106]])
Matrix([[0.12325993841442

Matrix([[0.122269939282405], [0.0555857611964298], [-0.0621539536549732]])
Matrix([[0.122259939292520], [0.0556576703018053], [-0.0622343597716959]])
Matrix([[0.122249939302654], [0.0557295794071062], [-0.0623147658883351]])
Matrix([[0.122239939312807], [0.0558014885123325], [-0.0623951720048910]])
Matrix([[0.122229939322978], [0.0558733976174844], [-0.0624755781213637]])
Matrix([[0.122219939333166], [0.0559453067225620], [-0.0625559842377532]])
Matrix([[0.122209939343372], [0.0560172158275653], [-0.0626363903540597]])
Matrix([[0.122199939353596], [0.0560891249324943], [-0.0627167964702832]])
Matrix([[0.122189939363837], [0.0561610340373493], [-0.0627972025864238]])
Matrix([[0.122179939374094], [0.0562329431421303], [-0.0628776087024817]])
Matrix([[0.122169939384368], [0.0563048522468373], [-0.0629580148184569]])
Matrix([[0.122159939394658], [0.0563767613514704], [-0.0630384209343495]])
Matrix([[0.122149939404965], [0.0564486704560297], [-0.0631188270501596]])
Matrix([[0.12213993941528

Matrix([[0.121159940458872], [0.0635676714499425], [-0.0710790321156573]])
Matrix([[0.121149940469377], [0.0636395805473023], [-0.0711594382234171]])
Matrix([[0.121139940479869], [0.0637114896445895], [-0.0712398443310957]])
Matrix([[0.121129940490348], [0.0637833987418041], [-0.0713202504386932]])
Matrix([[0.121119940500814], [0.0638553078389460], [-0.0714006565462094]])
Matrix([[0.121109940511266], [0.0639272169360152], [-0.0714810626536443]])
Matrix([[0.121099940521704], [0.0639991260330116], [-0.0715614687609978]])
Matrix([[0.121089940532128], [0.0640710351299351], [-0.0716418748682697]])
Matrix([[0.121079940542537], [0.0641429442267857], [-0.0717222809754602]])
Matrix([[0.121069940552932], [0.0642148533235633], [-0.0718026870825689]])
Matrix([[0.121059940563311], [0.0642867624202677], [-0.0718830931895960]])
Matrix([[0.121049940573675], [0.0643586715168991], [-0.0719634992965413]])
Matrix([[0.121039940584022], [0.0644305806134571], [-0.0720439054034046]])
Matrix([[0.12102994059435

Matrix([[0.120059941468893], [0.0714776717012377], [-0.0799237034568029]])
Matrix([[0.120049941476042], [0.0715495807898626], [-0.0800041095547957]])
Matrix([[0.120039941483140], [0.0716214898783971], [-0.0800845156526874]])
Matrix([[0.120029941490188], [0.0716933989668408], [-0.0801649217504775]])
Matrix([[0.120019941497185], [0.0717653080551934], [-0.0802453278481659]])
Matrix([[0.120009941504131], [0.0718372171434547], [-0.0803257339457521]])
